In [19]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split

model = ['nblogreg', 'nbnn', 'cnn', 'lstm']
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
PATH = '../../data/'

train_file = PATH + model[0] + '/train_' + model[0] + '.csv'
test_file = PATH + model[0] + '/' + model[0] + '.csv'
print(train_file)
print(test_file)

train = pd.read_csv(train_file)[label_cols]
test = pd.read_csv(test_file)[label_cols]

for i in range(1, len(model)):
    train_file = PATH + model[i] + '/train_' + model[i] + '.csv'
    test_file = PATH + model[i] + '/' + model[i] + '.csv'
    print(train_file)
    print(test_file)

    train = pd.concat([train, pd.read_csv(train_file)[label_cols]], axis=1)
    test = pd.concat([test, pd.read_csv(test_file)[label_cols]], axis=1)    

y = pd.read_csv(PATH + 'train.csv')[label_cols]

print('done')

../../data/nblogreg/train_nblogreg.csv
../../data/nblogreg/nblogreg.csv
../../data/nbnn/train_nbnn.csv
../../data/nbnn/nbnn.csv
      toxic  severe_toxic   obscene    threat    insult  identity_hate  \
0  0.010744      0.001672  0.003580  0.000165  0.004071       0.000924   
1  0.002284      0.000995  0.001188  0.000067  0.001180       0.000235   
2  0.031033      0.000498  0.021042  0.000348  0.013775       0.000553   
3  0.000497      0.000100  0.000873  0.000029  0.000385       0.000162   
4  0.017553      0.001465  0.018992  0.000101  0.025217       0.000806   

      toxic  severe_toxic   obscene        threat    insult  identity_hate  
0  0.000473  1.535056e-05  0.000264  1.213208e-06  0.000155       0.000015  
1  0.001564  9.673443e-05  0.000259  3.343771e-05  0.000162       0.000121  
2  0.016867  1.114338e-05  0.011044  6.280925e-05  0.001071       0.000096  
3  0.000063  2.725394e-07  0.000059  5.595627e-07  0.000015       0.000006  
4  0.032954  2.827321e-03  0.013184  5.560

In [22]:
X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.2, random_state=42)

out = np.zeros((test.shape[0], len(label_cols)))
for i, j in enumerate(label_cols):
    print('fit ' + j)
    ensemble = CatBoostClassifier(iterations=2,
                                  depth=10, 
                                  learning_rate=1, 
                                  loss_function='Logloss')
    ensemble.fit(X_train, y_train[j], use_best_model=True, eval_set=[X_test, y_test[j]], verbose=1)
    out[:, i] = ensemble.predict_proba(test.values)[:, 1] # TODO: should ues 1 or 0?

print('done')

fit toxic


/Users/haoyanzhai/Documents/work/venv/lib/python3.6/site-packages/catboost/core.py:533: FutureWarning: The 'verbose' parameter is deprecated, use 'logging_level' parameter instead (possible values: 'Silent', 'Verbose', 'Info', 'Debug').
  warnings.warn("The 'verbose' parameter is deprecated, use 'logging_level' parameter instead (possible values: 'Silent', 'Verbose', 'Info', 'Debug').", FutureWarning)


0:	learn: 0.0998597	test: 0.0991662	best: 0.0991662 (0)	total: 269ms	remaining: 269ms
1:	learn: 0.0962741	test: 0.0951578	best: 0.0951578 (1)	total: 588ms	remaining: 0us

bestTest = 0.09515779994
bestIteration = 1

Shrink model to first 2 iterations.
fit severe_toxic
0:	learn: 0.0232647	test: 0.0238527	best: 0.0238527 (0)	total: 307ms	remaining: 307ms
1:	learn: 0.0176526	test: 0.0198430	best: 0.0198430 (1)	total: 572ms	remaining: 0us

bestTest = 0.01984302806
bestIteration = 1

Shrink model to first 2 iterations.
fit obscene
0:	learn: 0.0538721	test: 0.0539248	best: 0.0539248 (0)	total: 301ms	remaining: 301ms
1:	learn: 0.0496647	test: 0.0497281	best: 0.0497281 (1)	total: 615ms	remaining: 0us

bestTest = 0.04972813464
bestIteration = 1

Shrink model to first 2 iterations.
fit threat
0:	learn: 0.0156799	test: 0.0128009	best: 0.0128009 (0)	total: 785ms	remaining: 785ms
1:	learn: 0.0162934	test: 0.0102531	best: 0.0102531 (1)	total: 1.18s	remaining: 0us

bestTest = 0.01025308027
bestIterati

In [23]:
submission = pd.read_csv(PATH + 'sample_submission.csv')
submission[label_cols] = out
submission.to_csv(PATH + 'ensemble/ensemble.csv', index=False)
print('done')

done
